In [12]:
# Prerequisites
import json
import os
from eye_ai.eye_ai import EyeAI

import pandas as pd
from pathlib import Path, PurePath
import logging

from deriva.core.ermrest_model import Model, Schema, Table, Column, Key, ForeignKey, builtin_types

from deriva_ml import DatasetSpec, FileSpec, ExecutionConfiguration, DerivaML, Workflow, VersionPart
from deriva_ml import MLVocab as vc
from deriva_ml.deriva_definitions import ColumnDefinition, BuiltinTypes
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

In [8]:
repo_dir = "/Users/vivi/Desktop/eye_ai/repos"   # Set this to be where your github repos are located.
%load_ext autoreload
%autoreload 2

# Update the load path so python can find modules for the model
import sys
from pathlib import Path
sys.path.insert(0, str(Path.home() / repo_dir / "deriva-ml"))
sys.path.insert(0, str(Path.home() / repo_dir / "eye-ai-ml"))

# Then you can import DerivaML
from deriva_ml import DerivaML, Workflow, ExecutionConfiguration, FileSpec
from deriva_ml import MLVocab as vc
from eye_ai.eye_ai import EyeAI

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
from deriva.core.utils.globus_auth_utils import GlobusNativeLogin

# host = 'www.eye-ai.org'
host = 'dev.eye-ai.org'
catalog_id = "1908"

gnl = GlobusNativeLogin(host=host)
if gnl.is_logged_in([host]):
    print("You are already logged in.")
else:
    gnl.login([host], no_local_server=True, no_browser=True, refresh_tokens=True, update_bdbag_keychain=True)
    print("Login Successful")

You are already logged in.


In [6]:
cache_dir = '/Users/vivi/Desktop/eye_ai/execution_cache'
working_dir = '/Users/vivi/Desktop/eye_ai/execution_cache'
EA = EyeAI(hostname = host, catalog_id = catalog_id, cache_dir= cache_dir, working_dir=working_dir)

In [5]:
c_domain = EA.catalog.getPathBuilder().schemas['eye-ai']
c_ml = EA.catalog.getPathBuilder().schemas['deriva-ml']

m_domain = EA.model.schemas['eye-ai']
m_ml = EA.model.schemas['deriva-ml']

In [10]:
def create_asso(table1, table2, schema):
    asso_def = Table.define_association([table1, table2])
    return schema.create_table(asso_def)
    
def create_file():
    table_def_file_type_vocab = Table.define_vocabulary(
        tname='File_Type', curie_template='eye-ai:{RID}'
    )
    file_type_table = m_ml.create_table(table_def_file_type_vocab)

    table_def_file = Table.define_asset(sname='deriva-ml', tname='File')
    file_table = m_ml.create_table(table_def_file)
    
    create_asso(file_table, file_type_table, m_ml)

In [11]:
# create_file()

In [9]:

filespec_iter = FileSpec.read_filespec(r"ai-ready-rfm-clean.json")
filespec_list = list(filespec_iter)[:5]
rids = EA.add_files(files=filespec_list, file_types=["RFM_Image"])

files = EA.list_files()
for f in files[:5]:
    print(f)

{'RID': '6-48PY', 'URL': 'tag:aireadi.eye-ai.org,2025:file:///data/aireadi/1f152774-13b8-4298-a6dc-cdc94bac3d94/dataset/CHANGELOG.md', 'MD5': 'b550f131cddfcd183e35c9360d6f5c02', 'Length': 2460, 'Description': '', 'File_Types': ['RFM_Image']}
{'RID': '6-48Q0', 'URL': 'tag:aireadi.eye-ai.org,2025:file:///data/aireadi/1f152774-13b8-4298-a6dc-cdc94bac3d94/dataset/LICENSE.txt', 'MD5': 'ce461cfc4e301e996c273dc50e82ed5f', 'Length': 8840, 'Description': '', 'File_Types': ['RFM_Image']}
{'RID': '6-48Q2', 'URL': 'tag:aireadi.eye-ai.org,2025:file:///data/aireadi/1f152774-13b8-4298-a6dc-cdc94bac3d94/dataset/README.md', 'MD5': '383699d710266f0ce88f6a9b87434d61', 'Length': 13804, 'Description': '', 'File_Types': ['RFM_Image']}
{'RID': '6-48Q4', 'URL': 'tag:aireadi.eye-ai.org,2025:file:///data/aireadi/1f152774-13b8-4298-a6dc-cdc94bac3d94/dataset/dataset_description.json', 'MD5': 'c1cf4261353a66b6560068faf80944fd', 'Length': 6121, 'Description': '', 'File_Types': ['RFM_Image']}
{'RID': '6-48Q6', 'URL'

In [18]:
vc.file_type.value

'File_Type'

In [ ]:
# m_ml.tables['File_File_Type'].drop()
# m_ml.tables['File_Type'].drop()
# m_ml.tables['File'].drop()